In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
!pip install transformers==4.15.0 sentencepiece
!pip install datasets==1.17.0 
!pip install langid
!pip install jiwer
! pip install pythainlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.4 MB 5.1 MB/s 
     |████████████████████████████████| 1.2 MB 51.0 MB/s 
     |████████████████████████████████| 596 kB 71.6 MB/s 
     |████████████████████████████████| 101 kB 12.4 MB/s 
     |████████████████████████████████| 3.3 MB 18.0 MB/s 
     |████████████████████████████████| 880 kB 59.3 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=bcca99528d4a80eebff6244d72b0b5c7df149a9ab1af5f688df30ac71f6d0b14
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |███

In [3]:
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
import re
from transformers import AutoModelForMaskedLM, pipeline
from transformers import AutoTokenizer, BertForTokenClassification
from tqdm import tqdm 
import pickle
from datasets import load_metric
import subprocess
import sys
import langid
from nltk.translate.gleu_score import sentence_gleu
from jiwer import cer
from pythainlp import correct

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
tokenizer = pickle.load(open('drive/MyDrive/AIBuilders/tpth/tokenizer_40k_nova.pkl', 'rb'))

In [10]:
ds_mlm = pickle.load(open('drive/MyDrive/AIBuilders/tpth/mlm_ds_40k_nova.pkl', 'rb'))
ds_tag = pickle.load(open('drive/MyDrive/AIBuilders/tpth/ner_ds_40k_nova.pkl', 'rb'))

In [11]:
# ds_mlm = ds_mlm[37890:]
# ds_tag = ds_tag[37890:]
# ds_mlm = ds_mlm[38890:]
# ds_tag = ds_tag[38890:]
# ds_mlm = ds_mlm[39890:]
# ds_tag = ds_tag[39890:]
ds_mlm = ds_mlm[40890:]
ds_tag = ds_tag[40890:]
NUM_SAMPLE = ds_mlm.shape[0]
NUM_SAMPLE

2000

In [12]:
bleu = load_metric("bleu")
glue = load_metric("glue", "mrpc")
totalacc = 0
bleu_higher = 0
bleu_lower = 0
bleu_equal = 0
total_f1 = 0
total_gs_ori = 0
total_gs_pred = 0
total_cer_ori = 0
total_cer_pred = 0

NUM_SAMPLE=1000
for i in tqdm(range(NUM_SAMPLE)):
  text_id = ds_tag.iloc[i]['text']['input_ids'].squeeze(0).tolist()
  text_id = [k for k in text_id if k != 1]
  text_token = tokenizer.convert_ids_to_tokens(text_id)
  original_token = text_token.copy()
  text_token_len = len(text_token)
  original = [original_token]
  labels_id = ds_mlm.iloc[i]['labels']
  labels_id = [k for k in labels_id if k != 1]
  labels_token = tokenizer.convert_ids_to_tokens(labels_id)
  references = [[labels_token]]
  # print(text_id)
  # print(labels_id)

  # print(f"NO. {i}, {''.join(text_token)}")

  for j in range(text_token_len):
    text_token[j] = text_token[j].replace("▁", " ")
    try:
      corrected = correct(text_token[j])
    except:
      print(f"err pythainlp: {text_token[j]}")
    else:
      # print(f"{text_token[j]} => {corrected}")
      text_token[j] = corrected
    
  predictions = [text_token]
  final = "".join(text_token)
  final = final.replace("▁", " ")

  numer = 0
  denom = 0
  TP = 0
  FP = 0
  TN = 0
  FN = 0
  # print(len(text_token), len(labels_token))
  # print(text_token)
  # print(labels_token)
  labels_len = len(labels_token)
  for i in range(labels_len):
    if not text_token[i] == original_token[i]: #change
      denom += 1
      if text_token[i] == labels_token[i]:
        numer += 1
        TP += 1
      elif not text_token[i] == labels_token[i]:
        FP += 1
    elif text_token[i] == original_token[i]: #no change
      if text_token[i] == labels_token[i]:
        TN += 1
      elif not text_token[i] == labels_token[i]:
        FN += 1
  if denom == 0:
    acc = 0
  else:
    acc = float(numer)/float(denom)
  totalacc += acc

  # print(f"TP:{TP}   TN:{TN}   FP:{FP}   FN:{FN}")
  precision = float(TP) / float(TP+FP) if TP+FP > 0 else 0
  recall = float(TP) / float(TP+FN) if TP+FN > 0 else 0
  f1 = float(2*precision*recall) / float(precision + recall) if precision+recall > 0 else 0
  total_f1 += f1

  references[0][0] = [k.replace("▁", " ") for k in references[0][0]]
  original[0] = [k.replace("▁", " ") for k in original[0]]
  predictions[0] = [k.replace("▁", " ") for k in predictions[0]]

  bleu_original = bleu.compute(predictions=original, references=references)
  bleu_prediction = bleu.compute(predictions=predictions, references=references)

  if bleu_prediction['bleu'] > bleu_original['bleu']:
    bleu_higher += 1
  elif bleu_prediction['bleu'] < bleu_original['bleu']:
    bleu_lower += 1
  elif bleu_prediction['bleu'] == bleu_original['bleu']:
    bleu_equal += 1

  # print(references)
  # print(predictions)
  # print(original)
  gleu_score = sentence_gleu(references[0], predictions[0], min_len=1, max_len=4)
  total_gs_pred += gleu_score
  gleu_score_original = sentence_gleu(references[0], original[0], min_len=1, max_len=4)
  total_gs_ori += gleu_score_original

  cer_text_ori = "".join(original[0])
  cer_text_pred = "".join(predictions[0])
  cer_text_ref = "".join(references[0][0])
  cer_text_ori = cer_text_ori.replace("_", "")
  cer_text_ori = cer_text_ori.replace("▁", "")
  cer_text_pred = cer_text_pred.replace("_", "")
  cer_text_pred = cer_text_pred.replace("▁", "")
  cer_text_ref = cer_text_ref.replace("_", "")
  cer_text_ref = cer_text_ref.replace("▁", "")
  # print(f"ORI: {cer_text_ori}")
  # print(f"PRED: {cer_text_pred}")
  # print(f"REF: {cer_text_ref}")
  cer_original = cer(cer_text_ori, cer_text_ref)
  total_cer_ori += cer_original
  cer_pred = cer(cer_text_pred, cer_text_ref)
  total_cer_pred += cer_pred

  # print(f"GLUE PREDICTED: {gleu_score}")
  # print(f"GLUE ORIGINAL: {gleu_score_original}")
  # print(f"CER PREDICTED: {cer_pred}")
  # print(f"CER ORIGINAL: {cer_original}")
  # print(f"ACC: {acc}")
  # print(f"f1: {f1}")
  # print(f"ORIGINAL : {bleu_original}")
  # print(f"PREDICTED: {bleu_prediction}")

  # print("\n-----------------------")
print(f"AVG ACC: {float(totalacc/NUM_SAMPLE)}")
print(f"AVG f1: {float(total_f1/NUM_SAMPLE)}")
print(f"# HIGHER BLEU PREDICTION: {bleu_higher}")
print(f"# LOWER BLEU PREDICTION: {bleu_lower}")
print(f"# EQUAL BLEU PREDICTION: {bleu_equal}")
print(f"# GLEU PREDICTION: {float(total_gs_pred/NUM_SAMPLE)}")
print(f"# GLEU ORIGINAL: {float(total_gs_ori/NUM_SAMPLE)}")
print(f"# CER PREDICTION: {float(total_cer_pred/NUM_SAMPLE)}")
print(f"# CER ORIGINAL: {float(total_cer_ori/NUM_SAMPLE)}")

100%|██████████| 1000/1000 [4:29:59<00:00, 16.20s/it]

AVG ACC: 0.0016464283215952699
AVG f1: 0.0029028534659916896
# HIGHER BLEU PREDICTION: 0
# LOWER BLEU PREDICTION: 1000
# EQUAL BLEU PREDICTION: 0
# GLEU PREDICTION: 0.2380370204838368
# GLEU ORIGINAL: 0.8641163716651621
# CER PREDICTION: 0.2457286122856353
# CER ORIGINAL: 0.0358968238745513


In [8]:
bleu = load_metric("bleu")
glue = load_metric("glue", "mrpc")
totalacc = 0
bleu_higher = 0
bleu_lower = 0
bleu_equal = 0
total_f1 = 0
total_gs_ori = 0
total_gs_pred = 0
total_cer_ori = 0
total_cer_pred = 0

NUM_SAMPLE=1000
for i in tqdm(range(NUM_SAMPLE)):
  text_id = ds_tag.iloc[i]['text']['input_ids'].squeeze(0).tolist()
  text_id = [k for k in text_id if k != 1]
  text_token = tokenizer.convert_ids_to_tokens(text_id)
  original_token = text_token.copy()
  text_token_len = len(text_token)
  original = [original_token]
  labels_id = ds_mlm.iloc[i]['labels']
  labels_id = [k for k in labels_id if k != 1]
  labels_token = tokenizer.convert_ids_to_tokens(labels_id)
  references = [[labels_token]]
  # print(text_id)
  # print(labels_id)

  # print(f"NO. {i}, {''.join(text_token)}")

  for j in range(text_token_len):
    text_token[j] = text_token[j].replace("▁", " ")
    try:
      corrected = correct(text_token[j])
    except:
      print(f"err pythainlp: {text_token[j]}")
    else:
      # print(f"{text_token[j]} => {corrected}")
      text_token[j] = corrected
    
  predictions = [text_token]
  final = "".join(text_token)
  final = final.replace("▁", " ")

  numer = 0
  denom = 0
  TP = 0
  FP = 0
  TN = 0
  FN = 0
  # print(len(text_token), len(labels_token))
  # print(text_token)
  # print(labels_token)
  labels_len = len(labels_token)
  for i in range(labels_len):
    if not text_token[i] == original_token[i]: #change
      denom += 1
      if text_token[i] == labels_token[i]:
        numer += 1
        TP += 1
      elif not text_token[i] == labels_token[i]:
        FP += 1
    elif text_token[i] == original_token[i]: #no change
      if text_token[i] == labels_token[i]:
        TN += 1
      elif not text_token[i] == labels_token[i]:
        FN += 1
  if denom == 0:
    acc = 0
  else:
    acc = float(numer)/float(denom)
  totalacc += acc

  # print(f"TP:{TP}   TN:{TN}   FP:{FP}   FN:{FN}")
  precision = float(TP) / float(TP+FP) if TP+FP > 0 else 0
  recall = float(TP) / float(TP+FN) if TP+FN > 0 else 0
  f1 = float(2*precision*recall) / float(precision + recall) if precision+recall > 0 else 0
  total_f1 += f1

  references[0][0] = [k.replace("▁", " ") for k in references[0][0]]
  original[0] = [k.replace("▁", " ") for k in original[0]]
  predictions[0] = [k.replace("▁", " ") for k in predictions[0]]

  bleu_original = bleu.compute(predictions=original, references=references)
  bleu_prediction = bleu.compute(predictions=predictions, references=references)

  if bleu_prediction['bleu'] > bleu_original['bleu']:
    bleu_higher += 1
  elif bleu_prediction['bleu'] < bleu_original['bleu']:
    bleu_lower += 1
  elif bleu_prediction['bleu'] == bleu_original['bleu']:
    bleu_equal += 1

  # print(references)
  # print(predictions)
  # print(original)
  gleu_score = sentence_gleu(references[0], predictions[0], min_len=1, max_len=4)
  total_gs_pred += gleu_score
  gleu_score_original = sentence_gleu(references[0], original[0], min_len=1, max_len=4)
  total_gs_ori += gleu_score_original

  cer_text_ori = "".join(original[0])
  cer_text_pred = "".join(predictions[0])
  cer_text_ref = "".join(references[0][0])
  cer_text_ori = cer_text_ori.replace("_", "")
  cer_text_ori = cer_text_ori.replace("▁", "")
  cer_text_pred = cer_text_pred.replace("_", "")
  cer_text_pred = cer_text_pred.replace("▁", "")
  cer_text_ref = cer_text_ref.replace("_", "")
  cer_text_ref = cer_text_ref.replace("▁", "")
  # print(f"ORI: {cer_text_ori}")
  # print(f"PRED: {cer_text_pred}")
  # print(f"REF: {cer_text_ref}")
  cer_original = cer(cer_text_ori, cer_text_ref)
  total_cer_ori += cer_original
  cer_pred = cer(cer_text_pred, cer_text_ref)
  total_cer_pred += cer_pred

  # print(f"GLUE PREDICTED: {gleu_score}")
  # print(f"GLUE ORIGINAL: {gleu_score_original}")
  # print(f"CER PREDICTED: {cer_pred}")
  # print(f"CER ORIGINAL: {cer_original}")
  # print(f"ACC: {acc}")
  # print(f"f1: {f1}")
  # print(f"ORIGINAL : {bleu_original}")
  # print(f"PREDICTED: {bleu_prediction}")

  # print("\n-----------------------")
print(f"AVG ACC: {float(totalacc/NUM_SAMPLE)}")
print(f"AVG f1: {float(total_f1/NUM_SAMPLE)}")
print(f"# HIGHER BLEU PREDICTION: {bleu_higher}")
print(f"# LOWER BLEU PREDICTION: {bleu_lower}")
print(f"# EQUAL BLEU PREDICTION: {bleu_equal}")
print(f"# GLEU PREDICTION: {float(total_gs_pred/NUM_SAMPLE)}")
print(f"# GLEU ORIGINAL: {float(total_gs_ori/NUM_SAMPLE)}")
print(f"# CER PREDICTION: {float(total_cer_pred/NUM_SAMPLE)}")
print(f"# CER ORIGINAL: {float(total_cer_ori/NUM_SAMPLE)}")

Downloading:   0%|          | 0.00/2.49k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.55k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.86k [00:00<?, ?B/s]

100%|██████████| 1000/1000 [4:14:02<00:00, 15.24s/it]

AVG ACC: 0.0010222687402040953
AVG f1: 0.0018401107401775825
# HIGHER BLEU PREDICTION: 0
# LOWER BLEU PREDICTION: 1000
# EQUAL BLEU PREDICTION: 0
# GLEU PREDICTION: 0.24594468942708955
# GLEU ORIGINAL: 0.8738663666530905
# CER PREDICTION: 0.23535208944018957
# CER ORIGINAL: 0.03306011655733898


In [ ]:
bleu = load_metric("bleu")
glue = load_metric("glue", "mrpc")
totalacc = 0
bleu_higher = 0
bleu_lower = 0
bleu_equal = 0
total_f1 = 0
total_gs_ori = 0
total_gs_pred = 0
total_cer_ori = 0
total_cer_pred = 0

NUM_SAMPLE=1000
for i in tqdm(range(NUM_SAMPLE)):
  text_id = ds_tag.iloc[i]['text']['input_ids'].squeeze(0).tolist()
  text_id = [k for k in text_id if k != 1]
  text_token = tokenizer.convert_ids_to_tokens(text_id)
  original_token = text_token.copy()
  text_token_len = len(text_token)
  original = [original_token]
  labels_id = ds_mlm.iloc[i]['labels']
  labels_id = [k for k in labels_id if k != 1]
  labels_token = tokenizer.convert_ids_to_tokens(labels_id)
  references = [[labels_token]]
  # print(text_id)
  # print(labels_id)

  # print(f"NO. {i}, {''.join(text_token)}")

  for j in range(text_token_len):
    text_token[j] = text_token[j].replace("▁", " ")
    try:
      corrected = correct(text_token[j])
    except:
      print(f"err pythainlp: {text_token[j]}")
    else:
      # print(f"{text_token[j]} => {corrected}")
      text_token[j] = corrected
    
  predictions = [text_token]
  final = "".join(text_token)
  final = final.replace("▁", " ")

  numer = 0
  denom = 0
  TP = 0
  FP = 0
  TN = 0
  FN = 0
  # print(len(text_token), len(labels_token))
  # print(text_token)
  # print(labels_token)
  labels_len = len(labels_token)
  for i in range(labels_len):
    if not text_token[i] == original_token[i]: #change
      denom += 1
      if text_token[i] == labels_token[i]:
        numer += 1
        TP += 1
      elif not text_token[i] == labels_token[i]:
        FP += 1
    elif text_token[i] == original_token[i]: #no change
      if text_token[i] == labels_token[i]:
        TN += 1
      elif not text_token[i] == labels_token[i]:
        FN += 1
  if denom == 0:
    acc = 0
  else:
    acc = float(numer)/float(denom)
  totalacc += acc

  # print(f"TP:{TP}   TN:{TN}   FP:{FP}   FN:{FN}")
  precision = float(TP) / float(TP+FP) if TP+FP > 0 else 0
  recall = float(TP) / float(TP+FN) if TP+FN > 0 else 0
  f1 = float(2*precision*recall) / float(precision + recall) if precision+recall > 0 else 0
  total_f1 += f1

  references[0][0] = [k.replace("▁", " ") for k in references[0][0]]
  original[0] = [k.replace("▁", " ") for k in original[0]]
  predictions[0] = [k.replace("▁", " ") for k in predictions[0]]

  bleu_original = bleu.compute(predictions=original, references=references)
  bleu_prediction = bleu.compute(predictions=predictions, references=references)

  if bleu_prediction['bleu'] > bleu_original['bleu']:
    bleu_higher += 1
  elif bleu_prediction['bleu'] < bleu_original['bleu']:
    bleu_lower += 1
  elif bleu_prediction['bleu'] == bleu_original['bleu']:
    bleu_equal += 1

  # print(references)
  # print(predictions)
  # print(original)
  gleu_score = sentence_gleu(references[0], predictions[0], min_len=1, max_len=4)
  total_gs_pred += gleu_score
  gleu_score_original = sentence_gleu(references[0], original[0], min_len=1, max_len=4)
  total_gs_ori += gleu_score_original

  cer_text_ori = "".join(original[0])
  cer_text_pred = "".join(predictions[0])
  cer_text_ref = "".join(references[0][0])
  cer_text_ori = cer_text_ori.replace("_", "")
  cer_text_ori = cer_text_ori.replace("▁", "")
  cer_text_pred = cer_text_pred.replace("_", "")
  cer_text_pred = cer_text_pred.replace("▁", "")
  cer_text_ref = cer_text_ref.replace("_", "")
  cer_text_ref = cer_text_ref.replace("▁", "")
  # print(f"ORI: {cer_text_ori}")
  # print(f"PRED: {cer_text_pred}")
  # print(f"REF: {cer_text_ref}")
  cer_original = cer(cer_text_ori, cer_text_ref)
  total_cer_ori += cer_original
  cer_pred = cer(cer_text_pred, cer_text_ref)
  total_cer_pred += cer_pred

  # print(f"GLUE PREDICTED: {gleu_score}")
  # print(f"GLUE ORIGINAL: {gleu_score_original}")
  # print(f"CER PREDICTED: {cer_pred}")
  # print(f"CER ORIGINAL: {cer_original}")
  # print(f"ACC: {acc}")
  # print(f"f1: {f1}")
  # print(f"ORIGINAL : {bleu_original}")
  # print(f"PREDICTED: {bleu_prediction}")

  # print("\n-----------------------")
print(f"AVG ACC: {float(totalacc/NUM_SAMPLE)}")
print(f"AVG f1: {float(total_f1/NUM_SAMPLE)}")
print(f"# HIGHER BLEU PREDICTION: {bleu_higher}")
print(f"# LOWER BLEU PREDICTION: {bleu_lower}")
print(f"# EQUAL BLEU PREDICTION: {bleu_equal}")
print(f"# GLEU PREDICTION: {float(total_gs_pred/NUM_SAMPLE)}")
print(f"# GLEU ORIGINAL: {float(total_gs_ori/NUM_SAMPLE)}")
print(f"# CER PREDICTION: {float(total_cer_pred/NUM_SAMPLE)}")
print(f"# CER ORIGINAL: {float(total_cer_ori/NUM_SAMPLE)}")

100%|██████████| 1000/1000 [4:59:35<00:00, 17.98s/it]

AVG ACC: 0.0018666903330733196
AVG f1: 0.0033239966347243612
# HIGHER BLEU PREDICTION: 0
# LOWER BLEU PREDICTION: 1000
# EQUAL BLEU PREDICTION: 0
# GLEU PREDICTION: 0.23835110303842347
# GLEU ORIGINAL: 0.8552652205795028
# CER PREDICTION: 0.241142016109713
# CER ORIGINAL: 0.03876608703880578


In [ ]:
bleu = load_metric("bleu")
glue = load_metric("glue", "mrpc")
totalacc = 0
bleu_higher = 0
bleu_lower = 0
bleu_equal = 0
total_f1 = 0
total_gs_ori = 0
total_gs_pred = 0
total_cer_ori = 0
total_cer_pred = 0

NUM_SAMPLE=1000
for i in tqdm(range(NUM_SAMPLE)):
  text_id = ds_tag.iloc[i]['text']['input_ids'].squeeze(0).tolist()
  text_id = [k for k in text_id if k != 1]
  text_token = tokenizer.convert_ids_to_tokens(text_id)
  original_token = text_token.copy()
  text_token_len = len(text_token)
  original = [original_token]
  labels_id = ds_mlm.iloc[i]['labels']
  labels_id = [k for k in labels_id if k != 1]
  labels_token = tokenizer.convert_ids_to_tokens(labels_id)
  references = [[labels_token]]
  # print(text_id)
  # print(labels_id)

  # print(f"NO. {i}, {''.join(text_token)}")

  for j in range(text_token_len):
    text_token[j] = text_token[j].replace("▁", " ")
    try:
      corrected = correct(text_token[j])
    except:
      print(f"err pythainlp: {text_token[j]}")
    else:
      # print(f"{text_token[j]} => {corrected}")
      text_token[j] = corrected
    
  predictions = [text_token]
  final = "".join(text_token)
  final = final.replace("▁", " ")

  numer = 0
  denom = 0
  TP = 0
  FP = 0
  TN = 0
  FN = 0
  # print(len(text_token), len(labels_token))
  # print(text_token)
  # print(labels_token)
  labels_len = len(labels_token)
  for i in range(labels_len):
    if not text_token[i] == original_token[i]: #change
      denom += 1
      if text_token[i] == labels_token[i]:
        numer += 1
        TP += 1
      elif not text_token[i] == labels_token[i]:
        FP += 1
    elif text_token[i] == original_token[i]: #no change
      if text_token[i] == labels_token[i]:
        TN += 1
      elif not text_token[i] == labels_token[i]:
        FN += 1
  if denom == 0:
    acc = 0
  else:
    acc = float(numer)/float(denom)
  totalacc += acc

  # print(f"TP:{TP}   TN:{TN}   FP:{FP}   FN:{FN}")
  precision = float(TP) / float(TP+FP) if TP+FP > 0 else 0
  recall = float(TP) / float(TP+FN) if TP+FN > 0 else 0
  f1 = float(2*precision*recall) / float(precision + recall) if precision+recall > 0 else 0
  total_f1 += f1

  references[0][0] = [k.replace("▁", " ") for k in references[0][0]]
  original[0] = [k.replace("▁", " ") for k in original[0]]
  predictions[0] = [k.replace("▁", " ") for k in predictions[0]]

  bleu_original = bleu.compute(predictions=original, references=references)
  bleu_prediction = bleu.compute(predictions=predictions, references=references)

  if bleu_prediction['bleu'] > bleu_original['bleu']:
    bleu_higher += 1
  elif bleu_prediction['bleu'] < bleu_original['bleu']:
    bleu_lower += 1
  elif bleu_prediction['bleu'] == bleu_original['bleu']:
    bleu_equal += 1

  # print(references)
  # print(predictions)
  # print(original)
  gleu_score = sentence_gleu(references[0], predictions[0], min_len=1, max_len=4)
  total_gs_pred += gleu_score
  gleu_score_original = sentence_gleu(references[0], original[0], min_len=1, max_len=4)
  total_gs_ori += gleu_score_original

  cer_text_ori = "".join(original[0])
  cer_text_pred = "".join(predictions[0])
  cer_text_ref = "".join(references[0][0])
  cer_text_ori = cer_text_ori.replace("_", "")
  cer_text_ori = cer_text_ori.replace("▁", "")
  cer_text_pred = cer_text_pred.replace("_", "")
  cer_text_pred = cer_text_pred.replace("▁", "")
  cer_text_ref = cer_text_ref.replace("_", "")
  cer_text_ref = cer_text_ref.replace("▁", "")
  # print(f"ORI: {cer_text_ori}")
  # print(f"PRED: {cer_text_pred}")
  # print(f"REF: {cer_text_ref}")
  cer_original = cer(cer_text_ori, cer_text_ref)
  total_cer_ori += cer_original
  cer_pred = cer(cer_text_pred, cer_text_ref)
  total_cer_pred += cer_pred

  # print(f"GLUE PREDICTED: {gleu_score}")
  # print(f"GLUE ORIGINAL: {gleu_score_original}")
  # print(f"CER PREDICTED: {cer_pred}")
  # print(f"CER ORIGINAL: {cer_original}")
  # print(f"ACC: {acc}")
  # print(f"f1: {f1}")
  # print(f"ORIGINAL : {bleu_original}")
  # print(f"PREDICTED: {bleu_prediction}")

  # print("\n-----------------------")
print(f"AVG ACC: {float(totalacc/NUM_SAMPLE)}")
print(f"AVG f1: {float(total_f1/NUM_SAMPLE)}")
print(f"# HIGHER BLEU PREDICTION: {bleu_higher}")
print(f"# LOWER BLEU PREDICTION: {bleu_lower}")
print(f"# EQUAL BLEU PREDICTION: {bleu_equal}")
print(f"# GLEU PREDICTION: {float(total_gs_pred/NUM_SAMPLE)}")
print(f"# GLEU ORIGINAL: {float(total_gs_ori/NUM_SAMPLE)}")
print(f"# CER PREDICTION: {float(total_cer_pred/NUM_SAMPLE)}")
print(f"# CER ORIGINAL: {float(total_cer_ori/NUM_SAMPLE)}")

100%|██████████| 1000/1000 [5:08:09<00:00, 18.49s/it]

AVG ACC: 0.001659097849941958
AVG f1: 0.002984204275700828
# HIGHER BLEU PREDICTION: 0
# LOWER BLEU PREDICTION: 1000
# EQUAL BLEU PREDICTION: 0
# GLEU PREDICTION: 0.23970478794255046
# GLEU ORIGINAL: 0.8636914705294098
# CER PREDICTION: 0.23985341176107164
# CER ORIGINAL: 0.035971571261440315


In [ ]:
class BertModel(torch.nn.Module):
    def __init__(self):
        super(BertModel, self).__init__()
        self.bert = BertForTokenClassification.from_pretrained('airesearch/wangchanberta-base-att-spm-uncased', num_labels=2)
        self.bert.resize_token_embeddings(len(tokenizer))

    def forward(self, input_id, mask, label):
        output = self.bert(input_ids=input_id, attention_mask=mask, labels=label, return_dict=False)
        return output

FILE = "drive/MyDrive/AIBuilders/tpth/tagging_tpth_200.pth"
tagging_model = BertModel()
tagging_model.load_state_dict(torch.load(FILE, map_location=torch.device('cpu')))
tagging_model.eval()

Downloading:   0%|          | 0.00/546 [00:00<?, ?B/s]

You are using a model of type camembert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.


Downloading:   0%|          | 0.00/404M [00:00<?, ?B/s]

Some weights of the model checkpoint at airesearch/wangchanberta-base-att-spm-uncased were not used when initializing BertForTokenClassification: ['roberta.encoder.layer.3.attention.output.LayerNorm.weight', 'roberta.encoder.layer.11.attention.output.LayerNorm.bias', 'roberta.encoder.layer.6.attention.output.LayerNorm.weight', 'roberta.encoder.layer.9.intermediate.dense.weight', 'lm_head.bias', 'roberta.encoder.layer.4.attention.output.dense.bias', 'roberta.encoder.layer.3.attention.output.dense.weight', 'roberta.encoder.layer.8.attention.self.query.weight', 'roberta.encoder.layer.6.attention.self.key.weight', 'roberta.encoder.layer.5.attention.output.dense.bias', 'roberta.encoder.layer.6.attention.self.value.bias', 'roberta.encoder.layer.10.output.LayerNorm.bias', 'roberta.encoder.layer.2.intermediate.dense.weight', 'roberta.encoder.layer.7.intermediate.dense.weight', 'roberta.encoder.layer.8.output.LayerNorm.weight', 'roberta.encoder.layer.2.output.LayerNorm.bias', 'roberta.encoder.l

BertModel(
  (bert): BertForTokenClassification(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(33660, 768)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(1, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias=True)
                (Lay

In [ ]:
ids_to_labels = {0: 'f', 1: 'i'}

def evaluate_one_text(model, sentence, mask, labels):
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    if use_cuda:
        model = model.cuda()

    input_id = torch.Tensor([sentence]).type(torch.int64)
    label_ids = []
    for i in sentence:
      if i == 1 or i == 5 or i == 6:
        label_ids.append(-100)
      else:
        label_ids.append(2)
    label_ids = torch.Tensor([label_ids]).type(torch.int64)
    mask = torch.Tensor([mask]).type(torch.int64)

    logits = tagging_model(input_id, mask, None)
    logits_clean = logits[0][label_ids != -100]

    predictions = logits_clean.argmax(dim=1).tolist()
    prediction_label = [ids_to_labels[i] for i in predictions]
    return prediction_label

In [ ]:
bleu = load_metric("bleu")
totalacc = 0
bleu_higher = 0
bleu_lower = 0
bleu_equal = 0
total_f1 = 0
total_gs_ori = 0
total_gs_pred = 0
total_cer_ori = 0
total_cer_pred = 0

# NUM_SAMPLE=10
for i in tqdm(range(NUM_SAMPLE)):
  text_id = ds_tag.iloc[i]['text']['input_ids'].squeeze(0).tolist()
  text_id = [k for k in text_id if k != 1]
  text_token = tokenizer.convert_ids_to_tokens(text_id)
  original_token = text_token.copy()
  text_token_len = len(text_token)
  original = [original_token]
  labels_id = ds_mlm.iloc[i]['labels']
  labels_id = [k for k in labels_id if k != 1]
  mask = ds_mlm.iloc[i]['attention_mask']
  mask = [k for k in mask if k != 0]
  labels_token = tokenizer.convert_ids_to_tokens(labels_id)
  references = [[labels_token]]

  i_f = evaluate_one_text(tagging_model, text_id, mask, labels_id)
  # print(i_f)
  i_f_len = len(i_f)

  # print(f"NO. {i}, {''.join(text_token)}")

  # print(f"TEXT TOKEN: {text_token}")
  for j in range(i_f_len):
    text_token[j+1] = text_token[j+1].replace("▁", " ")
    if(i_f[j] == 'i'):    
      try:
        corrected = correct(text_token[j+1])
      except:
        print(f"err pythainlp: {text_token[j+1]}")
      else:
        # print(f"{text_token[j+1]} => {corrected}")
        text_token[j+1] = corrected
  predictions = [text_token]
  final = "".join(text_token)
  final = final.replace("▁", " ")

  numer = 0
  denom = 0
  TP = 0
  FP = 0
  TN = 0
  FN = 0
  # print(len(text_token), len(labels_token))
  # print(text_token)
  # print(labels_token)
  labels_len = len(labels_token)
  for i in range(labels_len):
    if not text_token[i] == original_token[i]: #change
      denom += 1
      if text_token[i] == labels_token[i]:
        numer += 1
        TP += 1
      elif not text_token[i] == labels_token[i]:
        FP += 1
    elif text_token[i] == original_token[i]: #no change
      if text_token[i] == labels_token[i]:
        TN += 1
      elif not text_token[i] == labels_token[i]:
        FN += 1
  if denom == 0:
    acc = 0
  else:
    acc = float(numer)/float(denom)
  totalacc += acc

  # print(f"TP:{TP}   TN:{TN}   FP:{FP}   FN:{FN}")
  precision = float(TP) / float(TP+FP) if TP+FP > 0 else 0
  recall = float(TP) / float(TP+FN) if TP+FN > 0 else 0
  f1 = float(2*precision*recall) / float(precision + recall) if precision+recall > 0 else 0
  total_f1 += f1

  references[0][0] = [k.replace("▁", " ") for k in references[0][0]]
  original[0] = [k.replace("▁", " ") for k in original[0]]
  predictions[0] = [k.replace("▁", " ") for k in predictions[0]]

  # print(references)
  # print(original)
  # print(predictions)

  bleu_original = bleu.compute(predictions=original, references=references)
  bleu_prediction = bleu.compute(predictions=predictions, references=references)

  if bleu_prediction['bleu'] > bleu_original['bleu']:
    bleu_higher += 1
  elif bleu_prediction['bleu'] < bleu_original['bleu']:
    bleu_lower += 1
  elif bleu_prediction['bleu'] == bleu_original['bleu']:
    bleu_equal += 1

  gleu_score = sentence_gleu(references[0], predictions[0], min_len=1, max_len=4)
  total_gs_pred += gleu_score
  gleu_score_original = sentence_gleu(references[0], original[0], min_len=1, max_len=4)
  total_gs_ori += gleu_score_original

  cer_text_ori = "".join(original[0])
  cer_text_pred = "".join(predictions[0])
  cer_text_ref = "".join(references[0][0])
  cer_text_ori = cer_text_ori.replace("_", "")
  cer_text_ori = cer_text_ori.replace("▁", "")
  cer_text_pred = cer_text_pred.replace("_", "")
  cer_text_pred = cer_text_pred.replace("▁", "")
  cer_text_ref = cer_text_ref.replace("_", "")
  cer_text_ref = cer_text_ref.replace("▁", "")
  # print(f"ORI: {cer_text_ori}")
  # print(f"PRED: {cer_text_pred}")
  # print(f"REF: {cer_text_ref}")
  cer_original = cer(cer_text_ori, cer_text_ref)
  total_cer_ori += cer_original
  cer_pred = cer(cer_text_pred, cer_text_ref)
  total_cer_pred += cer_pred

  # print(f"GLUE PREDICTED: {gleu_score}")
  # print(f"GLUE ORIGINAL: {gleu_score_original}")
  # print(f"CER PREDICTED: {cer_pred}")
  # print(f"CER ORIGINAL: {cer_original}")
  # print(f"ACC: {acc}")
  # print(f"f1: {f1}")
  # print(f"ORIGINAL : {bleu_original}")
  # print(f"PREDICTED: {bleu_prediction}")

  # print("\n-----------------------")
print(f"AVG ACC: {float(totalacc/NUM_SAMPLE)}")
print(f"AVG f1: {float(total_f1/NUM_SAMPLE)}")
print(f"# HIGHER BLEU PREDICTION: {bleu_higher}")
print(f"# LOWER BLEU PREDICTION: {bleu_lower}")
print(f"# EQUAL BLEU PREDICTION: {bleu_equal}")
print(f"# GLEU PREDICTION: {float(total_gs_pred/NUM_SAMPLE)}")
print(f"# GLEU ORIGINAL: {float(total_gs_ori/NUM_SAMPLE)}")
print(f"# CER PREDICTION: {float(total_cer_pred/NUM_SAMPLE)}")
print(f"# CER ORIGINAL: {float(total_cer_ori/NUM_SAMPLE)}")

100%|██████████| 5000/5000 [45:18<00:00,  1.84it/s]

AVG ACC: 0.0019727217347858456
AVG f1: 0.0030288659576171734
# HIGHER BLEU PREDICTION: 103
# LOWER BLEU PREDICTION: 2099
# EQUAL BLEU PREDICTION: 2798
# GLEU PREDICTION: 0.8487772656839223
# GLEU ORIGINAL: 0.8660844114008937
# CER PREDICTION: 0.04070377522416636
# CER ORIGINAL: 0.03541724276517062
